In [7]:
# 유투브에서 검색 후 동영상의 리뷰 정보 수집하기
#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver

import time
import sys
import re
import math
import numpy 
import pandas as pd   
import xlwt 
import random
import os
#import openpyxl
# import urllib.request
# import urllib

#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *80)
print("유투브 영상의 댓글을 수집하는 웹크롤러입니다")
print("=" *80)

query_txt=input("1.유튜브에서 검색할 주제 키워드를 입력하세요(예:올리브영): ")

movie_cnt = int(input('2.위 주제로 댓글을 크롤링할 유튜브 영상은 몇건입니까?:'))
reple_cnt = int(input('3.각 동영상에서 추출할 댓글은 몇건입니까?: '))

f_dir = input("4.크롤링 결과를 저장할 폴더명만 쓰세요(기본값:c:\\temp\\):")
if f_dir == '' :
    f_dir = 'c:\\temp\\'

print("\n")
print("요청하신 정보로 데이터를 수집중이니 잠시만 기다려 주세요~~^^")   

 # Step 3. 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+'youtube'+'-'+s+'-'+query_txt)
os.chdir(f_dir+'youtube'+'-'+s+'-'+query_txt)

ff_dir=f_dir+'youtube'+'-'+s+'-'+query_txt
ff_name=f_dir+'youtube'+'-'+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+'youtube'+'-'+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+'youtube'+'-'+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

s_time = time.time( )

# Step 4. 크롬 드라이버 설정하고 유투브 사이트에 접속하여 검색을 진행합니다 
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
driver= webdriver.Chrome()

driver.get('https://www.youtube.com')

time.sleep(5)
element = driver.find_element_by_name("search_query")
element.send_keys(query_txt)
element.submit()
time.sleep(2)

# Step 5. 검색할 동영상의 URL 정보를 수집하여 대상 영상 목록을 만듭니다.
# 검색 결과가 첫 화면에 20개가 나옵니다.
# 만약 사용자가 입력한 동영상 개수가 20개가 넘어갈 경우 스크롤다운해서 갯수를
# 먼저 만든 후 URL 을 추출하여 URL 을 완성해야 합니다

def scroll_down(driver):
    #driver.execute_script("window.scrollTo(500,document.body.scrollHeight);")
    driver.execute_script("window.scrollBy(0,800);") # 한페이지 20개씩 출력값
    time.sleep(5)

if movie_cnt < 20 :
    page_cnt = 1 
else :
    page_cnt = math.ceil(cnt/20)

if movie_cnt > 20 :
    i = 1
    while ( i <=  page_cnt) :
        print("화면을 %s 회 아래로 스크롤 합니다" %i)
        scroll_down(driver)
        time.sleep(1)
        i += 1

# 검색된 유튜브 영상의 URL 을 추출합니다

time.sleep(2)

html = driver.page_source
soup1 = BeautifulSoup(html, 'html.parser')

count = 0
item=[]

print("\n")

for i in soup1.find_all('a','yt-simple-endpoint style-scope ytd-video-renderer'):
    item.append(i['href'])
    count += 1

    if count == movie_cnt :
        break

print("검색된 영상 중 %s 건 동영상의 댓글을 추출하겠습니다" %movie_cnt)
print("\n")

# Step 6. 동영상의 상세 정보를 수집합니다.
# 수집된 전체 URL 중에서 사용자가 입력한 수만큼의 동영상의 댓글을 수집합니다.
# 먼저 전체 URL 을 완성 -> 동영상 페이지 열기 -> 댓글 수집 -> 다음 동영상 순으로 진행합니다
full_url=[]
url_cnt = 0
for x in range(0,len(item)) :
    url_cnt += 1
    url = 'https://www.youtube.com/' +item[x]
    full_url.append(url)
    if url_cnt == movie_cnt :
        break

play_cnt = 1

url2=[]
reple2=[]
reple3=[]
reple4=[]
writer2=[]
wdate2=[]
good2=[]
bad2=[]

real_collect_count = 0   #실제 총 수집한 리플 갯수
# 비트맵 이미지 아이콘을 위한 대체 딕셔너리를 만든다
bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

for y in range(0,len(full_url)) :
    driver.get(full_url[y])
    time.sleep(10)
    print("\n")
    print("\n")
    print("%s 번째 동영상의 정보를 수집합니다." %play_cnt)

    i = 1
    while ( i <=  page_cnt) :
        scroll_down(driver)
        time.sleep(3)
        i += 1

    html = driver.page_source
    soup2 = BeautifulSoup(html, 'html.parser')
    time.sleep(5)
    # 조회수
    t_count_0 = soup2.select('#count')
    t_count_1 = t_count_0[1].get_text()
    t_count_2 = t_count_1.replace(",","")
    t_count_3 = re.search("\d+",t_count_2)
    t_count_4 = int(t_count_3.group())
    
    # 제목
    t_title_1 = soup2.select('#info-contents')
    t_title_2 = t_title_1[0].find('h1').get_text()
    t_title_3 = t_title_2.translate(bmp_map).replace("\n","")
    
    # 댓글수
    t_view_1 = soup2.find('yt-formatted-string','count-text style-scope ytd-comments-header-renderer').get_text()#.get_text( ).replace("\n","")
    t_view_2 = t_view_1.replace("," ,"")
    t_view_3 = re.search("\d+",t_view_2)
    t_view_4 = int(t_view_3.group())
    
    #좋아요 / 싫어요 수
    t_good_bad = soup2.select('#top-level-buttons-computed')
    t_good = t_good_bad[0].select('#text')[0].get_text()  # 좋아요 수
    t_bad = t_good_bad[0].select('#text')[1].get_text()   # 싫어요 수

    print("=" *120)
    print("%s 번째 동영상의 제목은 %s 입니다" %(play_cnt,t_title_3))
    print("%s 번째 동영상의 조회수는 %s 회, 댓글수는 총 %s개, 좋아요는 %s개, 싫어요는 %s개입니다" %(play_cnt,t_count_4,t_view_4,t_good,t_bad))  
    print("=" *120)
    print("\n")
        
    #댓글 정보를 수집하기 위해 화면을 스크롤해서 아래로 이동합니다
    if t_view_4 > 0 :
        print("%s 번째 동영상에서 댓글 수집을 시작합니다" %play_cnt)
        print("댓글의 개수가 많아서 시간이 걸릴 수 있으니 잠시 기다려 주세요~~^^")
        print("\n")
        if reple_cnt < 20 :
            reple_page_cnt = 1
        else :
            reple_page_cnt = math.ceil(reple_cnt / 20)

        i = 1
        while ( i <  reple_page_cnt) :
            print("%s 번째 페이지의 댓글을 수집하는 중입니다~" %i)
            scroll_down(driver)
            time.sleep(0.5)
            i += 1

        #내용을 수집합니다

        time.sleep(2)

        html = driver.page_source
        soup3 = BeautifulSoup(html, 'html.parser')

        count = 0

        reple_result = soup3.select('#comments > #sections > #contents')

        for a in range(0,reple_cnt+1) :
            real_collect_count += 1
            count += 1

            f = open(ff_name, 'a',encoding='UTF-8')
            f.write("%s 번째 영상의 %s 번째  댓글 " %(play_cnt,count))
            f.write("------------------------------------------------------------------"+"\n")

            # 댓글 작성자
            try : 
                writer = reple_result[0].select(" #author-text")[a].get_text().replace("\n","").strip()
            except  :
                break
            else :
                print("\n")
                print("%s 번째 영상의 %s 번째  댓글 " %(play_cnt,count))
                print("-" *70)

                # 유튜브 URL 주소
                print("1.URL 주소:",full_url[y])
                f.write("1.유투브 URL주소: " + full_url[y] + "\n")
                url2.append(full_url[y])

                print("2.댓글 작성자명:", writer)
                f.write("2.댓글 작성자명: " + writer + "\n")
                writer2.append(writer)

                # 댓글 작성일자
                wdate = reple_result[0].select("yt-formatted-string.published-time-text > a.yt-simple-endpoint")[a].get_text().replace("\n","")

                print("3.댓글 작성일자:",wdate)
                f.write("3.댓글 작성일자: " + wdate + "\n")
                wdate2.append(wdate)

                # 댓글 내용
                reple1 = reple_result[0].select('#expander > #content > #content-text')[a].get_text().replace("\n","")

                reple2=reple1.translate(bmp_map).replace("\n","")
                print("4.댓글 내용:",reple2)
                f.write("4.댓글 내용: " + reple2 + "\n")
                reple3.append(reple2)

                f.write("\n")

                #좋아요 / 싫어요
                good2.append(t_good)
                bad2.append(t_bad)
                
            f.close( )

            if count == reple_cnt :
                break

        time.sleep(2)       
        play_cnt += 1  
    else :
        print("%s 번째 동영상의 댓글수가 %s개이므로 다음영상으로 넘어갑니다" %(play_cnt,t_view_4) )
        f = open(ff_name, 'a',encoding='UTF-8')
        f.write("%s 번째 영상의 %s 번째  댓글 " %(play_cnt,count))
        f.write("------------------------------------------------------------------"+"\n")
        f.write("%s 번째 동영상의 댓글수가 %s개이므로 다음영상으로 넘어갑니다" %(play_cnt,t_view_4) )
        f.write("\n")
        f.close( )
        
        play_cnt += 1 

 #step 9. 엑셀 형태로 저장하기
                
youtube_reple = pd.DataFrame()
youtube_reple['URL 주소']=url2
youtube_reple['댓글작성자명']=pd.Series(writer2)
youtube_reple['댓글작성일자']=pd.Series(wdate2)
youtube_reple['댓글 내용']=pd.Series(reple3)
youtube_reple['좋아요수']=pd.Series(good2)
youtube_reple['싫어요수']=pd.Series(bad2)

# csv 형태로 저장하기
youtube_reple.to_csv(fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장하기
youtube_reple.to_excel(fx_name ,index=True)

e_time = time.time( )
t_time = e_time - s_time

real_collect_count -= 1

# txt 파일에 크롤링 요약 정보 저장하기
orig_stdout = sys.stdout
f = open(ff_name, 'a',encoding='UTF-8')
sys.stdout = f

print("\n")
print("=" *50)
print("총 소요시간은 %s 초 이며," %t_time)
print("총 저장 건수는 %s 건 입니다 " %real_collect_count)
print("=" *50)

sys.stdout = orig_stdout
f.close( )

print("\n") 
print("=" *100)
print("1.요청된 총 %s 건 동영상에서 각 영상별 실제 크롤링 된 리뷰수는 총 %s 건입니다" %(movie_cnt,real_collect_count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *100)

driver.close( )

유투브 영상의 댓글을 수집하는 웹크롤러입니다
1.유튜브에서 검색할 주제 키워드를 입력하세요(예:올리브영): 손흥민
2.위 주제로 댓글을 크롤링할 유튜브 영상은 몇건입니까?:3
3.각 동영상에서 추출할 댓글은 몇건입니까?: 3
4.크롤링 결과를 저장할 폴더명만 쓰세요(기본값:c:\temp\):


요청하신 정보로 데이터를 수집중이니 잠시만 기다려 주세요~~^^


검색된 영상 중 3 건 동영상의 댓글을 추출하겠습니다






1 번째 동영상의 정보를 수집합니다.
1 번째 동영상의 제목은 손흥민 토트넘 재계약 임박? “훈련장 복귀 후 발표” 입니다
1 번째 동영상의 조회수는 4244 회, 댓글수는 총 272개, 좋아요는 252개, 싫어요는 12개입니다


1 번째 동영상에서 댓글 수집을 시작합니다
댓글의 개수가 많아서 시간이 걸릴 수 있으니 잠시 기다려 주세요~~^^




1 번째 영상의 1 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=0Vvlb6PA9aY
2.댓글 작성자명: 칠판을 1등급 맞게한 현우진
3.댓글 작성일자: 2시간 전
4.댓글 내용: 골드는 레비의 손흥민 압박용이라 상관 안 써도 될듯


1 번째 영상의 2 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=0Vvlb6PA9aY
2.댓글 작성자명: Pass Master Toni Kroos
3.댓글 작성일자: 2시간 전
4.댓글 내용: 재계약을 하든 말든 손흥민 선수 항상 응원합니다 Sonny � � � � �� �� � �


1 번째 영상의 3 번째  댓글 
--------------------------------------------------------